In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
winemag = pd.read_csv('./winemag-data-130k-v2.csv', index_col=0)

In [4]:
winemag.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [5]:
winemag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


In [6]:
winemag.isna().sum()

country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64

In [7]:
len(winemag['region_1'].unique())

1230

In [8]:
X = pd.get_dummies(winemag[['country', 'price', 'variety']])
y = winemag['points']
X.columns

Index(['price', 'country_Argentina', 'country_Armenia', 'country_Australia',
       'country_Austria', 'country_Bosnia and Herzegovina', 'country_Brazil',
       'country_Bulgaria', 'country_Canada', 'country_Chile',
       ...
       'variety_Yapincak', 'variety_Zelen', 'variety_Zibibbo',
       'variety_Zierfandler', 'variety_Zierfandler-Rotgipfler',
       'variety_Zinfandel', 'variety_Zlahtina', 'variety_Zweigelt',
       'variety_Çalkarası', 'variety_Žilavka'],
      dtype='object', length=751)

In [9]:
X.isna().sum()

price                8996
country_Argentina       0
country_Armenia         0
country_Australia       0
country_Austria         0
                     ... 
variety_Zinfandel       0
variety_Zlahtina        0
variety_Zweigelt        0
variety_Çalkarası       0
variety_Žilavka         0
Length: 751, dtype: int64

In [10]:
X['price'].describe()

count    120975.000000
mean         35.363389
std          41.022218
min           4.000000
25%          17.000000
50%          25.000000
75%          42.000000
max        3300.000000
Name: price, dtype: float64

In [11]:
X['price'] = X['price'].fillna(X['price'].mean()).round()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [13]:
d_tree_classifier = DecisionTreeClassifier(random_state=1)
d_tree_classifier.fit(X_train, y_train)
predict_tree_default = d_tree_classifier.predict(X_val)

In [14]:
mean_absolute_error(y_val, predict_tree_default)

2.06977303327563

optimization

In [15]:
def get_mae(max_leaf_nodes):
    d_tree_classifier = DecisionTreeClassifier(random_state=1, max_leaf_nodes=max_leaf_nodes)
    d_tree_classifier.fit(X_train, y_train)
    predict_tree_default = d_tree_classifier.predict(X_val)
    return mean_absolute_error(y_val, predict_tree_default)

In [16]:
[get_mae(x) for x in [5, 50, 500, 5000]]

[2.0540007693787268,
 1.9464800923254473,
 1.9445566455087517,
 2.0020196191575304]

500 max_leaf_nodes performed better.
Now let's test.

In [17]:
d_tree_classifier_test = DecisionTreeClassifier(random_state=1, max_leaf_nodes=500)
d_tree_classifier_test.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))
predict_tree_test = d_tree_classifier_test.predict(X_test)
mean_absolute_error(y_test, predict_tree_test)

1.9393344874014233

In [18]:
random_forest_classifier_test = RandomForestClassifier(random_state=1, n_estimators=10)
random_forest_classifier_test.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))
predict_random_forest_test = random_forest_classifier_test.predict(X_test)
mean_absolute_error(y_test, predict_random_forest_test)

2.049394114252741

In [19]:
from xgboost import XGBClassifier
xgb_classifier_test = XGBClassifier(random_state=1, n_estimators=10)
xgb_classifier_test.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))
predict_xgb_test = xgb_classifier_test.predict(X_test)
mean_absolute_error(y_test, predict_xgb_test)

1.9980765531833045